# `컨볼루션 신경망의 기초와 PyTorch에서의 AlexNet 모델 실습 적용`

![DMLPCNN](./pic/DMLPCNN.png)

![conv](./pic/conv.png)

![stridepadding](./pic/stridepadding.png)

![pooling](./pic/pooling.png)

![str](./pic/str.png)

- 실습에서 살펴볼 `AlexNet 모델`은 2012년도에 높은 정확도 개선을 보여준 대표적인 모델이다.

- `8개의 계층 구조`
- `ReLU 함수`를 사용한 첫 모델
- fully connected layer층에 `drop out 기법`을 사용
- `max pooling 기법` 사용

In [ ]:
import torch
import torch.nn as nn
from .utils import load_state_dict_from_url
from typing import Any


__all__ = ['AlexNet', 'alexnet']


model_urls = {
    'alexnet': 'https://download.pytorch.org/models/alexnet-owt-4df8aa71.pth',
}


class AlexNet(nn.Module):

    def __init__(self, num_classes: int = 1000) -> None:
        super(AlexNet, self).__init__()

        # nn.Sequential : layer 쌓는 곳 (5개의 conv와 3개의 pooling)
        # 모델의 특징 추출 부분에 해당함
        self.features = nn.Sequential(

            #Conv1
            #input channel : 3 , output channel : 64, kernel_size : 11*11, stride : 4, padding : 2
            
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True), # inplace=True 하면, inplace 연산을 수행함, inplace 연산은 결과값을 새로운 변수에 값을 저장하는 대신 기존의 데이터를 대체하는것을 의미
            #Max Pool1
            nn.MaxPool2d(kernel_size=3, stride=2),
            #Conv2 이젠 아웃풋 채널과 동일한 인풋 64
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            #Max Pool2
            nn.MaxPool2d(kernel_size=3, stride=2),
            #Conv3 
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            ##Conv4
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            #Conv5
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            #Max Pool3
            nn.MaxPool2d(kernel_size=3, stride=2),
        )

         # average pooling 연산 진행
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        
        # 모델의 분류증에 해당함
        self.classifier = nn.Sequential(

            #드롭아웃 
            nn.Dropout(),
            # fully connected layer1 : 차원 256*6*6 아웃풋 4096
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),

            # fully connected layer2 : 이전 아웃풋 = 인픗 4096
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),

            # fully connected layer3 (최정 분류층)
            nn.Linear(4096, num_classes),
        )

    # 이미지 데이터 (x)가 들어왔을 때 거치는 layer들 선언
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # 위 특징 추출 부분 layer
        x = self.features(x)
       
        # average pooling 부분 layer
        x = self.avgpool(x)
        #output shape : (batch size * 256(channel), 6, 6)
        
        # Flatten layer : 격자 모양의 특징맵을 한 줄로 펼쳐주는 layer
        x = torch.flatten(x, 1)
        #output shape (batch_size, 256 * 6* 6)
        
        # 분류 layer 
        x = self.classifier(x)
        return x

# 1. 
# pytorch에서 구현될 모델 함수(alexnet)를 호출할 때 사용하는 def
# pretrained : 사전 학습된 모델을 가져올지말지 결정하는 불리안 인자
def alexnet(pretrained: bool = False, progress: bool = True, **kwargs: Any) -> AlexNet:
    r"""AlexNet model architecture from the
    `"One weird trick..." <https://arxiv.org/abs/1404.5997>`_ paper.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """

    # 모델에 알렉스넷 모델 함수를 넣어주면ㅅ 시작한다!
    model = AlexNet(**kwargs)
    if pretrained:
        state_dict = load_state_dict_from_url(model_urls['alexnet'],
                                              progress=progress)
        model.load_state_dict(state_dict)
    return model